# Advent of Code 2023 - Day 10: **Pipe Maze**

### Solved with ray-casting, only for part 1

In [44]:
from collections import defaultdict
import numpy as np
with open('input.txt', 'r') as f:
    data = [line.split(" ") for line in f.read().splitlines()]

In [45]:
class Edge:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
    def __repr__(self):
        return f"E({self.start}, {self.end})"
        
class Pos:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __add__(self, other):
        return Pos(self.x + other.x, self.y + other.y)
    
    def __mul__(self, other):
        return Pos(self.x * other, self.y * other)
    
    def __repr__(self):
        return f"P({self.x:>2}, {self.y:>2})"
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self) -> int:
        return hash((self.x, self.y))

In [46]:
edges = defaultdict(list)
outline = set()

dir_map = {
    'R' : Pos(1, 0),
    'L' : Pos(-1, 0),
    'U' : Pos(0, -1),
    'D' : Pos(0, 1)
}

boarders = {
    'min_x' : 0,
    'max_x' : 0,
    'min_y' : 0,
    'max_y' : 0
}

cur = Pos(0, 0)
for line in data:
    move, length = line[0], int(line[1])
    new_edge = Edge(cur, cur + dir_map[move] * length)
    if new_edge.start.x == new_edge.end.x:
        edges[new_edge.start.x].append(new_edge)
    cur = new_edge.end
    
    boarders = {
        'min_x' : min(boarders['min_x'], cur.x),
        'max_x' : max(boarders['max_x'], cur.x),
        'min_y' : min(boarders['min_y'], cur.y),
        'max_y' : max(boarders['max_y'], cur.y)
    }
    
    for x in range(min(new_edge.start.x, new_edge.end.x), max(new_edge.start.x, new_edge.end.x) + 1):
        for y in range(min(new_edge.start.y, new_edge.end.y), max(new_edge.start.y, new_edge.end.y) + 1):
            outline.add(Pos(x, y))
    
# move into positive space
edges = {k + abs(boarders['min_x']) : v for k, v in edges.items()}
edges = {k : [Edge(e.start + Pos(abs(boarders['min_x']), abs(boarders['min_y'])), e.end + Pos(abs(boarders['min_x']), abs(boarders['min_y']))) for e in v] for k, v in edges.items()}
outline = {Pos(p.x + abs(boarders['min_x']), p.y + abs(boarders['min_y'])) for p in outline}

In [47]:
grid = np.zeros((boarders['max_x'] - boarders['min_x'] + 1, boarders['max_y'] - boarders['min_y'] + 1), dtype=np.int8)

for y in range(0, - boarders['min_y'] + boarders['max_y'] + 1):
    counter = 0
    u_counter = 0
    d_counter = 0
    for x in range(0, - boarders['min_x'] + boarders['max_x'] + 1):
        if x not in edges:
            grid[x][y] = (counter + u_counter if u_counter == d_counter else counter) % 2
            if Pos(x, y) in outline:
                grid[x][y] = 1
            # print(y, x, counter, u_counter, d_counter, now_count)
            continue
        for edge in edges[x]:
            if edge.start.y <= y <= edge.end.y or edge.end.y <= y <= edge.start.y:
                if y == edge.start.y and edge.end.y < edge.start.y or y == edge.end.y and edge.start.y < edge.end.y:
                    d_counter = (d_counter + 1) % 2
                elif y == edge.end.y and edge.end.y < edge.start.y or y == edge.start.y and edge.start.y < edge.end.y:
                    u_counter  = (u_counter + 1) % 2
                else:
                    counter += 1
        now_count = (counter + u_counter if u_counter == d_counter else counter) % 2
        # print(y, x, counter, u_counter, d_counter, now_count)
        grid[x][y] = now_count
        if Pos(x, y) in outline:
                grid[x][y] = 1
        
grid.T
            
# safe grid to file
with open('grid.txt', 'w') as f:
    for y in range(0, - boarders['min_y'] + boarders['max_y'] + 1):
        for x in range(0, - boarders['min_x'] + boarders['max_x'] + 1):
            f.write(str(grid[x][y]))
        f.write('\n')

In [48]:
grid.sum()

40745